In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LassoCV, RidgeCV, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline
sns.set(rc={'figure.figsize': [10, 10]}, font_scale=1.2)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/housesalesprediction/kc_house_data.csv')
df

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(col + '\n------')
    print(df[col].value_counts())
    print('---------------------')

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
stand= StandardScaler()

In [ ]:

for column in ['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15']:
    df[column] = stand.fit_transform(df[column].values.reshape(-1,1))

In [ ]:
df1=df.drop('date', axis = 1)
plt.figure(figsize=(16,16))
sns.heatmap(df1.corr(), annot=True, fmt=".1f");

In [ ]:
#Let's split the dataset
x = df1.drop('price', axis=1)
y = df1[['price']]


In [ ]:
#Modeling
#It's regression problem because the target variable ('price') is continuous variable.

def fitting_models(x, y, cv=3):
    names = []
    scores_mse = []
    scores_rmse = []
    scores_r2 = []
    
    regressors = [
        ('LinearRegression', LinearRegression()),
        ('DecisionTreeRegressor', DecisionTreeRegressor()),
        ('RandomForestRegressor', RandomForestRegressor()),
        ('GradientBoostingRegressor', GradientBoostingRegressor()),
        ('KNeighborsRegressor', KNeighborsRegressor()),
        ('SVR', SVR()),
        ('Lasso', Lasso()),
        ('Ridge', Ridge())
    ]
    
    for model in regressors:
        names.append(model[0])
        
        cv_results = cross_validate(model[1], x, y, cv=cv, scoring=('r2', 'neg_mean_squared_error'))

        mse = -cv_results['test_neg_mean_squared_error'].mean()
        r2_score = cv_results['test_r2'].mean()
        rmse = np.sqrt(mse)
        
        scores_mse.append(mse)
        scores_rmse.append(rmse)
        scores_r2.append(r2_score)
        
        print(f"Model's name: {model[0]}: MSE -> {mse}; RMSE -> {rmse}; 'R2' -> {r2_score}")
        
    
    data = pd.DataFrame({
        'names':names,
        'mse':scores_mse,
        'rmse':scores_rmse,
        'r2':scores_r2
    })
    
    return data

data = fitting_models(x, y)

print(data)
